[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/nlp.ipynb)

# NLP: Zpracování přirozeného jazyka

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

Cílem tohoto cvičení je provést základní kvantitativní textovou analýzu některého digitalizovaného dokumentu z **Archivu Jana Patočky** ([AJP](https://archiv.janpatocka.cz/items/browse?tags=fulltext)). Omezíme se však pouze na dokumenty, u kterých je dostupný digitální přepis (tzv. fulltext). Tj. URL adresa, kterou hledáme, je adresa jakéhokoliv námi vybraného dokumentu z daného archivu pro který je dostupný přepis.

## Cvičení

In [ ]:
%%capture
!pip install stanza
!pip install gensim
import stanza
stanza.download("cs")
czech_pipeline = stanza.Pipeline("cs")
import requests
from urllib.request import urlopen # pro práci w webovými adresami
from bs4 import BeautifulSoup # pro práci s webovými stránkami ve formátu html
import pandas as pd # pro práci s tabulkami ve formátu dataframe
import nltk # modul pro práci s textovými daty
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
import matplotlib.pyplot as plt # modul pro vytváření grafů
import numpy as np # modul pro pokročilejší matematické operace
import re
from sklearn.decomposition import TruncatedSVD
from gensim.models import KeyedVectors


Váš hlavní úkol je spojen s buňkou níže. V ní je potřeba nahradit obsah proměnné "url", tj. **vyměnit webovou adresu jednoho dokumentu z AJP za adresu jiného dokumentu z téhož archivu**.  Pozor, že webová adresa musí být uvnitř uvozovek. Aby se změna projevila, je třeba buňku nakonec spustit.

In [ ]:
url = "https://archiv.janpatocka.cz/items/show/308"
web_text = urlopen(url).read().decode("utf-8")
soup = BeautifulSoup(web_text, "html.parser")

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="nlp", arg1=url)

In [ ]:
text_title = soup.find("div", id="item_title").get_text()
text_title

In [ ]:
[div for div in soup.find_all("div", class_="col span_7_of_9")][3].get_text()

In [ ]:
text_dokumentu = soup.find("div", id="trans_full").get_text()
text_dokumentu = " ".join(text_dokumentu.split())
print(text_dokumentu)

In [ ]:
# dokument jako list slov získáme pomocí funkce "split()"
# uložíme si ho takto do nové proměnné "string_list"
string_list = text_dokumentu.split()
# prvních 20 prvků tohoto listu si nyní vypíšeme:
string_list[:20]

In [ ]:
### pomocí funkce "len()" spočítáme délku tohoto listu slov:
len(string_list)

### Lematizace a postagging

S textem článku, tak jak se nyní nachází v proměnné "text_clanku", bychom se ale při kvantitativní textové analýze stále příliš daleko nedostali. Čeština je totiž morfologicky velice bohatý jazyk. Chceme-li např. spočítat kolikrát se v textu objevuje sloveso "mít", s textem v aktuální podobě se příliš daleko nedstaneme. Zde potřebujeme na naše textová data aplikovat dvě další procedury:


1.   lemmatizace, tj. převedení slov z textu do jejich základních tvarů (slovesa do infinitivu, podstatná jména do 1.pádu singuláru apod.)
2.   POS-tagging ("part-of-speech tags"),  tj. určení slovních druhů a mluvnických kategorií

Aplikace těchto procedur nám umožní získat data z hlediska kvantitativní textové analýzy výrazně zajímavější.

V případě češtiny se můžeme v tomto případě opřít o model pro jazykový model pro zpracování češtiny vyvinutý pro knihovnu [stanza](https://stanfordnlp.github.io/stanza/), konkrétně [stanza-cs](https://huggingface.co/stanfordnlp/stanza-cs). 


In [ ]:
sent = "Česká republika je velmi sebevědomý stát, který dokáže stát na vlastních nohách."
doc = czech_pipeline(sent)
doc

In [ ]:
# vytvoříme morfoligicky zpracovanou verzi našeho textu a konvenčně si ji uložíme do proměnně `doc`.  
doc = czech_pipeline(text_dokumentu)

Tímto jsme vytvořili morfoligicky zpracovanou verzi našeho textu a konvenčně si ji uložili do proměnně `doc`. Tento objekt nyní neobsahuje pouze syrový text, ale také text rozdělený do vět, každou větu na jednotlivá slova a každému slovo je automaticky přiřazeno jeho *lemma*, morfologické určení a některé další atributy. Na tuto datovou strukturou se můžeme nyní pracovat např. následujícím způsobem:

In [ ]:
data = []
for i, sentence in enumerate(doc.sentences):
  for token in sentence.words:
    data.append({
      'sent_n' : i,
      'text': token.text,
      'lemma': token.lemma,
      'upos': token.upos,
      'xpos': token.xpos
    })
data_df = pd.DataFrame(data)
data_df[30:40]

Obdobně můžeme vybrat z vět pouze lemmata slov vybraných slovních druhů:

In [ ]:
lemmatized_sentences = []
for sent in doc.sentences:
  sentence_lemmata = []
  for token in sent.words:
    if token.upos in ["PROPN", "NOUN", "VERB", "ADJ"]:
      sentence_lemmata.append(token.lemma)
  lemmatized_sentences.append(sentence_lemmata)

In [ ]:
lemmatized_sentences[:3]

V této podobě může být již vcelku zajímavé podívat se na frekvence výskytů slov, resp. jejich lematizovaných tvarů. K tomu použijeme funkce z modulu "nltk".

In [ ]:
# nejprve si z naší filtrované tabulky vyxtrahujeme lemmata samotná
lemmata = [l for s in lemmatized_sentences for l in s]
# pro každý jednotlivý výraz necháme spočítat jeho počet výskytů
lemmata_freq = nltk.FreqDist(lemmata)
# vybereme např. 10 nejfrekventovanějších slov (rozumějme lemmatizovaných substantiv, adjektiv a sloves)
lemmata_most_freq = lemmata_freq.most_common(10)
print(lemmata_most_freq)

Nyní si tato data vizualizujeme.

In [ ]:
# kvůli horizontálnímu zobrazení prohodíme pořadí na našem listu
lemmata_mostfreq = lemmata_most_freq
lemmata_mostfreq.reverse()

# pro potřeby grafu přiřadíme hodnoty jednotlivým osám
height = [tup[1] for tup in lemmata_mostfreq]
bars = [tup[0] for tup in lemmata_mostfreq]
y_pos = np.arange(len(bars))

plt.barh(y_pos, height)
# graf si pojmenujeme a osu také
plt.yticks(y_pos, bars)
plt.xlabel('Frekvence výskytů')
plt.title('Frekvence výskytů nejčastějších slov')
# graf si zobrazíme
plt.show()

Přehled nejfrekventovanějších slov jistě představuje cennou informaci o obsahu zkoumaného textu či souboru textů. Neříká však nic o tom, jakým způsobem jsou zde příslušná slova použita. K tomu potřebujeme zaprvé a především informaci o tom, která slova se spolu a jak často spoluvyskytují. Tuto informaci získáme tak, že zkonstruujeme tzv. **kookurenční matici**. 



In [ ]:
vocabulary = [tup[0] for tup in lemmata_freq.most_common(100)]
vect = CountVectorizer(vocabulary=vocabulary, lowercase=False)
X = vect.fit_transform([" ".join(sent) for sent in lemmatized_sentences])
Xc = (X.T * X)  # This is the matrix manipulation step
cooccurrence_matrix_df = pd.DataFrame(Xc.todense(), index=vocabulary, columns=vocabulary)
cooccurrence_matrix_df.head()

In [ ]:
kontrola_pruchodu(ntb="nlp", arg1=vocabulary[:10])

Pokud jsme se dostali až jsem, hlavní část našeho úkolu byla splněna. Máme-li však čas a chuť, můžeme pokračovat dále.

> **Zde končí povinná část cvičení.**

## Rozšiřující analýza: Analýza všech děl z Archivu Jana Patočky 

In [ ]:
def get_patocka(url):
  textdata = {}
  try:
    web_text = urlopen(url).read().decode("utf-8")
    soup = BeautifulSoup(web_text, "html.parser")
    text_title = soup.find("div", id="item_title").get_text()
    date = [div for div in soup.find_all("div", class_="col span_7_of_9")][3].get_text()
    text_dokumentu = soup.find("div", id="trans_full").get_text()
    text_dokumentu = " ".join(text_dokumentu.split())
    #print(id)
    textdata["url"] = url
    textdata["title"] = text_title
    textdata["date"] = date
    textdata["rawtext"] = text_dokumentu
  except:
    pass
  return textdata

Získejme zdrojový kód jedné stránky soupisu přepsaných textů a získejme URL odkazy všech dokumentů, na které odkazuje: 

In [ ]:
base_url = "https://archiv.janpatocka.cz/items/browse?tags=fulltext&page="
page_url = base_url + str(2)
resp_text = requests.get(page_url).text
["https://archiv.janpatocka.cz" + href for href in re.findall("/items/show/\d+", resp_text)]

Tuto proceduru nyní zopakujeme pro všechny zbývající stránky:

In [ ]:
docs_urls = []
base_url = "https://archiv.janpatocka.cz/items/browse?tags=fulltext&page="
for page_n in range(1,34):
  page_url = base_url + str(page_n)
  resp_text = requests.get(page_url).text
  page_urls = ["https://archiv.janpatocka.cz" + href for href in re.findall("/items/show/\d+", resp_text)]
  docs_urls.extend(page_urls)

kolik odkazů jsme získali?

In [ ]:
len(docs_urls)

In [ ]:
docs_urls = list(set(docs_urls))
len(docs_urls)

In [ ]:
docs_urls[:3]

Nyní můžeme získat přepisy textů ze všech těchto stránek. Tato procedura však zabere značný čas. V rámci výuky ji tudíž můžeme přeskočit a načteme místo toho již předpřipravená data.

In [ ]:
#data = []
#for url in docs_urls:
#  textdata = get_patocka(url)
#  data.append(textdata)
#patocka_all_df = pd.DataFrame(data)
#patocka_all_df = patocka_all_df.dropna()

Textová data ve sloupci `rawtext` si nyní můžeme lemmatizovat pomocí funkce níže. I tato procedura je však v případě celého korpusu relativně výpočetně náročná, a proto ji nyní také přeskočíme a načteme již lemmatizovaná data.

In [ ]:
def get_lemmatized_sents(rawtext):
  try:
    doc = czech_pipeline(rawtext)
    lemmatized_sentences = []
    for sent in doc.sentences:
        sentence_lemmata = []
        for token in sent.words:
            if token.upos in ["PROPN", "NOUN", "VERB", "ADJ"]:
                sentence_lemmata.append(token.lemma)
        lemmatized_sentences.append(sentence_lemmata)
    return lemmatized_sentences
  except:
    return []

In [ ]:
#patocka_all_df["lemmatized_sents"] = patocka_all_df["rawtext"].apply(get_lemmatized_sents)
# patocka_all_df.to_json("../data/patocka_all_df.json")

In [ ]:
patocka_all_df = pd.read_json("https://raw.githubusercontent.com/CCS-ZCU/pribehy-dat/master/data/patocka_all_df.json")
patocka_all_df.head(5)

In [ ]:
patocka_all_df.shape

In [ ]:
all_sents = [sent for doc in patocka_all_df["lemmatized_sents"] for sent in doc]
docs = [" ".join(sent) for sent in all_sents]
lemmata_flat = [t for s in all_sents for t in s]
lemmata_freqs = nltk.FreqDist(lemmata_flat).most_common()
vocabulary = [tup[0] for tup in lemmata_freqs][:500]
lemmata_freqs[:10]

In [ ]:
vect = CountVectorizer(vocabulary=vocabulary, lowercase=False)
X = vect.fit_transform(docs)
Xc = (X.T * X)  # This is the matrix manipulation step
cooccurrence_matrix_df = pd.DataFrame(Xc.todense(), index=vocabulary, columns=vocabulary)
cooccurrence_matrix_df.head()

In [ ]:
svd = TruncatedSVD(n_components=100)  # choose the number of components
X_reduced = svd.fit_transform(Xc)

In [ ]:
kv = KeyedVectors(vector_size=X_reduced.shape[1])
for word, vector in zip(vocabulary, X_reduced):
  kv.add_vector(word, vector)

In [ ]:
kv.most_similar("svět")